In [1]:
# !pip install ydata_profiling

In [2]:
!pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 kB 1.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
# from ydata_profiling import ProfileReport
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
%matplotlib inline

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
football_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ML/Lab 1/football_data.csv', low_memory=False)

In [6]:
football_df.head(5)

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2016-12-11,2016121101,13,2,NaN,02:00,2,1920.0,5.0,SD,...,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,2016
1,2016-12-11,2016121101,13,2,1.0,02:00,2,1920.0,0.0,SD,...,0.330475,0.968025,0.031975,0.940170,0.059830,0.031975,0.027855,0.024299,0.003556,2016
2,2016-12-11,2016121101,13,2,1.0,01:50,2,1910.0,10.0,CAR,...,NaN,0.940170,0.059830,0.947575,0.052425,0.059830,-0.007405,NaN,NaN,2016
3,2016-12-11,2016121101,13,2,2.0,01:17,2,1877.0,33.0,CAR,...,-3.766159,0.947575,0.052425,0.953601,0.046399,0.052425,-0.006025,0.036136,-0.042161,2016
4,2016-12-11,2016121101,13,2,3.0,01:08,2,1868.0,9.0,CAR,...,0.000000,0.953601,0.046399,0.912354,0.087646,0.046399,0.041247,0.041247,0.000000,2016


In [7]:
types = football_df.dtypes
print("Number categorical featues:", sum(types=='object'))
print(types)

Number categorical featues: 38
Date         object
GameID        int64
Drive         int64
qtr           int64
down        float64
             ...   
Win_Prob    float64
WPA         float64
airWPA      float64
yacWPA      float64
Season        int64
Length: 102, dtype: object


In [8]:
football_df.describe()

,GameID,Drive,qtr,down,TimeUnder,TimeSecs,PlayTimeDiff,yrdln,yrdline100,ydstogo,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
count,1.000000e+04,10000.000000,10000.000000,8448.000000,10000.000000,9992.000000,9984.000000,9980.000000,9980.000000,10000.000000,...,3929.000000,9357.000000,9357.000000,9309.000000,9309.000000,10000.000000,9.845000e+03,3930.000000,3923.000000,10000.0
mean,2.016372e+09,12.444900,2.584500,1.991951,7.322400,1686.264011,20.739283,28.566433,47.859218,7.158400,...,-0.400785,0.547408,0.453126,0.547667,0.452708,0.470946,1.673133e-03,0.014596,-0.009764,2016.0
std,3.996451e+05,7.075561,1.131542,1.006399,4.685216,1064.437980,20.795996,12.533362,24.737534,4.844632,...,2.008014,0.300660,0.300744,0.302591,0.302650,0.319106,4.684680e-02,0.056720,0.065689,0.0
min,2.016121e+09,1.000000,1.000000,1.000000,0.000000,-849.000000,0.000000,1.000000,1.000000,0.000000,...,-12.156367,0.000000,0.000000,0.000000,0.000000,0.000000,-9.719480e-01,-0.943119,-0.939447,2016.0
25%,2.016122e+09,6.000000,2.000000,1.000000,3.000000,772.000000,5.000000,19.000000,29.000000,3.000000,...,-0.950799,0.298087,0.183309,0.295898,0.179571,0.172491,-1.375776e-02,-0.009758,-0.017638,2016.0
50%,2.016122e+09,12.000000,3.000000,2.000000,7.000000,1800.000000,18.000000,30.000000,48.000000,9.000000,...,0.000000,0.565156,0.435096,0.568334,0.431754,0.486653,-7.985673e-07,0.004151,0.000000,2016.0
75%,2.017010e+09,18.000000,4.000000,3.000000,11.000000,2575.000000,37.000000,38.000000,69.000000,10.000000,...,0.469628,0.817714,0.702465,0.820549,0.705156,0.750169,1.278280e-02,0.033391,0.009991,2016.0
max,2.017010e+09,30.000000,5.000000,4.000000,15.000000,3600.000000,908.000000,50.000000,99.000000,36.000000,...,8.079540,1.000000,1.000000,1.000000,1.000000,1.000000,9.739204e-01,0.898214,0.932194,2016.0


In [9]:
# report = ProfileReport(football_df)
# report.to_file('data_profile_report.html')

## 8. Self Practice task

Using the Football dataset follow the steps below:
1. Select `Win_Prob` as target variable
1. Remove columns with constant value (i.e all the values in the column are equal to zero)
1. Extract features from `Date` and `time` column
1. Remove all columns with more than 99% missing values
1. Remove all columns with 99% distinct values
1. Split the data into train (80%) and test(20%) sets.
1. Split the train data into train (80%) and validation sets.
1. Encode categorical data using a different encoder (not One Hot encoder) : see [Category Encoders](https://contrib.scikit-learn.org/category_encoders/index.html) for a full list. remember that you fit the encoder on train data only and then transform test data
1. Impute missing values

<span style="color:red"> NOTE : Make use of the insights from pandas profiling report</span>

In [10]:
# Select Win_Prob as target variable
y = football_df[['Win_Prob']]
football_df.drop(columns= ['Win_Prob'], inplace=True)

In [11]:
# Remove columns with constant value (i.e all the values in the column are equal to zero)
uniq_columns = [col for col in football_df if len(football_df[col].unique()) == 1]
football_df.drop(columns=uniq_columns, inplace=True)

In [12]:
# Extract features from Date and time column
# Convert 'Date' to datetime type and add new columns
football_df['Date'] = pd.to_datetime(football_df['Date'])
football_df['year'] = football_df['Date'].dt.year
football_df['month'] = football_df['Date'].dt.month
football_df['day'] = football_df['Date'].dt.day

# Convert 'time' to datetime type and add new columns
football_df['time'] = pd.to_datetime(football_df['time'])
football_df['hour'] = football_df['time'].dt.hour
football_df['minute'] = football_df['time'].dt.minute

# Dalete columns 'Date' и 'time'
football_df.drop(['Date', 'time'], axis=1, inplace=True)

In [13]:
# Remove all columns with more than 99% missing values
missing_values_columns = [col for col in football_df if (football_df[col].isna().sum() / len(football_df[col])) * 100 >= 99]
football_df.drop(columns=missing_values_columns, inplace=True)

In [14]:
# Remove all columns with 99% distinct values
distinct_values_columns = [col for col in football_df if (football_df[col].nunique() / len(football_df)) * 100 >= 99]
football_df.drop(columns=distinct_values_columns, inplace=True)

In [15]:
# Split the data into train (80%) and test(20%) sets.
X = football_df
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.8, random_state=42)

In [16]:
football_df.isnull().sum()

GameID          0
Drive           0
qtr             0
down         1552
TimeUnder       0
             ... 
year            0
month           0
day             0
hour            8
minute          8
Length: 96, dtype: int64

In [17]:
# Split the train data into train (80%) and validation sets.
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, train_size = 0.8, random_state=42)

In [18]:
import pandas as pd
import category_encoders as ce

categorical_columns = X_train.select_dtypes(include=['object']).columns

target_encoder = ce.TargetEncoder(cols=categorical_columns)

X_train_encoded = target_encoder.fit_transform(X_train, y_train)
X_test_encoded = target_encoder.transform(X_test)
X_valid_encoded = target_encoder.transform(X_valid)


In [19]:
# report = ProfileReport(X)
# report.to_file('data_profile_report.html')

In [20]:
categorical_columns = X_train_encoded.select_dtypes(include=['object']).columns

In [21]:
categorical_columns

Index([], dtype='object')

In [22]:
# Impute missing values
def count_nans(df):
    # 1 or 2 lines
    return np.isnan(df).sum().sum()

print("Before Imputing:")
print("#Empty missing data cells in the X_train_encoded = ", count_nans(X_train_encoded))
print("#Empty missing data cells in the X_test_encoded = ", count_nans(X_test_encoded))
print("#Empty missing data cells in the X_valid_encoded = ", count_nans(X_valid_encoded))

simple_imputer = SimpleImputer(strategy="mean")
X_train_encoded_imputed = simple_imputer.fit_transform(X_train_encoded)
X_test_encoded_imputed = simple_imputer.transform(X_test_encoded)
X_valid_encoded_imputed = simple_imputer.transform(X_valid_encoded)

print("After Imputing:")
print("#Empty missing data cells in the X_train_encoded = ", count_nans(X_train_encoded_imputed))
print("#Empty missing data cells in the X_test_encoded = ", count_nans(X_test_encoded_imputed))
print("#Empty missing data cells in the X_valid_encoded = ", count_nans(X_valid_encoded_imputed))

Before Imputing:
#Empty missing data cells in the X_train_encoded =  26953
#Empty missing data cells in the X_test_encoded =  8598
#Empty missing data cells in the X_valid_encoded =  6480
After Imputing:
#Empty missing data cells in the X_train_encoded =  0
#Empty missing data cells in the X_test_encoded =  0
#Empty missing data cells in the X_valid_encoded =  0
